In [1]:
import hopsworks
import pandas as pd
import joblib
import os
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2


In [2]:
fv_version=1
model_version=1
td_version=1

In [3]:
# key=""
# with open("api-key.txt", "r") as f:
#     key = f.read().rstrip()
# # os.environ['HOPSWORKS_PROJECT']="loan_approval"
# # os.environ['HOPSWORKS_HOST']="staging.cloud.hopsworks.ai"
# os.environ['HOPSWORKS_PROJECT']="deloitte"
# os.environ['HOPSWORKS_HOST']="6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai"
# os.environ['HOPSWORKS_API_KEY']=key

In [4]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
fv = fs.get_feature_view(name="loans_applicants", version=fv_version)

In [ ]:
fv.init_batch_scoring(training_dataset_version=td_version)

In [ ]:
def retrieve_data(feature_view):
    batch_data = feature_view.get_batch_data()
    return batch_data

df = retrieve_data(fv)
df

In [ ]:
def get_model(project):
    mr = project.get_model_registry()
    model = mr.get_model("lending_model", version=model_version)
    model_dir = model.download()
    return joblib.load(model_dir + "/lending_model.pkl")

model = get_model(proj)

In [ ]:
predictions = model.predict(df)
predictions

In [ ]:
np.unique(predictions, return_counts=True)

In [13]:
cols = []
for td in fv.schema:
    cols.append(td.name)

predictions = pd.DataFrame(numpy_array, columns=cols)

['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade', 'loan_status', 'purpose', 'zip_code', 'earliest_cr_line_year', 'home_ownership', 'annual_inc', 'verification_status', 'dti', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'application_type', 'mort_acc', 'pub_rec_bankruptcies']


In [ ]:
applicants_fg = fs.get_or_create_feature_group(name="applicants",
                                    version=1,
                                    description="Lending Club Loan Applicants",
                                    online_enabled=True,
                                    primary_key=['id'],
                                    partition_key=['earliest_cr_line_year'],
                                    event_time='earliest_cr_line'
)